In [3]:
import pandas as pd
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA, BollingerBands
from indicators import time_to_epoch, epoch_to_time
import datetime
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import requests

def get_market_rates(symbol:str, timeframe:str, previous_candle_count:int, eohlcv:bool=True, base_url:str="http://10.168.85.228:7767/data") -> np.ndarray:
    if timeframe not in ["1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"]:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')

    if timeframe == "1m":
        _tf = 1
    elif timeframe == "5m":
        _tf = 5
    elif timeframe == "15m":
        _tf = 15
    elif timeframe == "30m":
        _tf = 30
    elif timeframe == "1h":
        _tf = 16385
    elif timeframe == "1d":
        _tf = 16408
    elif timeframe == "4h":
        _tf = 16388
    elif timeframe == "1w":
        _tf = 32769
    elif timeframe == "1mo":
        _tf = 49153
    else:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')
    payload = {
        "symbol": symbol,
        "timeframe": _tf,
        "previous": previous_candle_count,
        "date_from": datetime.datetime.now()
    }

    response = requests.post(base_url, json=payload)

    if response.status_code == 200:
        _data = response.json()['rates']
        # print("Bars received:", len(_data))
        if eohlcv:
            return np.array(_data)[:, :6]
        else:
            return np.array(_data)
    else:
        print("Error:", response.text)
        raise RuntimeError(response.text)


In [5]:
data_1h = get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=10000)
data_1h.shape

TypeError: Object of type datetime is not JSON serializable

In [41]:
indx = 8000
[epoch_to_time(data_1h[indx, 0], timezone="UTC")] + data_1h[indx, 1:].tolist()

['2025-08-29 17:00:00', 3429.27, 3440.94, 3429.17, 3440.9, 5734.0]

In [38]:
data_4h = get_market_rates("XAUUSD", timeframe="4h", previous_candle_count=10000)
data_4h.shape

(10000, 6)

In [39]:
for indx, i in enumerate(data_4h[:, 0]):
    if i > data_1h[-2000, 0]:
        print(indx - 1)
        break

9476


In [40]:
indx = 9476
[epoch_to_time(data_4h[indx, 0], timezone="UTC")] + data_4h[indx, 1:].tolist()


['2025-08-29 16:00:00', 3416.75, 3449.38, 3413.69, 3449.17, 21362.0]

In [51]:
data_1d = get_market_rates("XAUUSD", timeframe="1d", previous_candle_count=50000)
data_1d.shape

(5538, 6)

In [52]:
for indx, i in enumerate(data_1d[:, 0]):
    if i > data_1h[-2000, 0]:
        print(indx - 1)
        break

5450


In [54]:
indx = 5450
[epoch_to_time(data_1d[indx, 0], timezone="UTC")] + data_1d[indx, 1:].tolist()


['2025-08-29 00:00:00', 3417.06, 3453.88, 3404.36, 3448.16, 90252.0]

In [162]:
main_ind = 13+57
indx_1d = 5450
indx_4h = 9476
indx_1h = 8000

current_epoch = data_1h[indx_1h+main_ind, 0]
current_time = epoch_to_time(current_epoch, timezone="UTC")
current_price = data_1h[indx_1h+main_ind, 4]
print(f"Current time is {current_time} and current price is {current_price}")


print("Following are the 100 previous candles on timeframe of 1 hour:")
for indx_1h, i in enumerate(data_1h[:, 0]):
    if i >= current_epoch:
        # print(indx_1h-1)
        indx_1h = indx_1h+1
        break
for i in data_1h[indx_1h-100:indx_1h].tolist():
    print(str(i)[1:-1])
# print(epoch_to_time(i[0], timezone="UTC"))
last_1h = i


print("Following are the 100 previous candles on timeframe of 4 hour:")
for indx_4h, i in enumerate(data_4h[:, 0]):
    if i >= current_epoch:
        # print(indx_4h-1)
        indx_4h = indx_4h-1
        break
for i in data_4h[indx_4h-100:indx_4h].tolist():
    print(str(i)[1:-1])
last_4h = i
# print(epoch_to_time(i[0], timezone="UTC"))



print("Following are the 100 previous candles on timeframe of 1 day:")
for indx_1d, i in enumerate(data_1d[:, 0]):
    if i >= current_epoch:
        indx_1d = indx_1d-1
        break

for i in data_1d[indx_1d-100:indx_1d].tolist():
    print(str(i)[1:-1])
# print(epoch_to_time(i[0], timezone="UTC"))
last_1d = i


Current time is 2025-09-03 20:00:00 and current price is 3574.29
Following are the 100 previous candles on timeframe of 1 hour:
1756378800.0, 3395.54, 3401.05, 3393.95, 3400.72, 4029.0
1756382400.0, 3400.8, 3401.56, 3398.47, 3399.26, 3271.0
1756386000.0, 3399.33, 3399.33, 3393.99, 3397.06, 3834.0
1756389600.0, 3397.03, 3407.7, 3396.23, 3404.08, 4224.0
1756393200.0, 3404.13, 3408.75, 3396.92, 3396.98, 5103.0
1756396800.0, 3397.16, 3410.36, 3394.59, 3407.4, 5535.0
1756400400.0, 3407.24, 3413.45, 3406.58, 3412.26, 5150.0
1756404000.0, 3412.23, 3413.17, 3404.54, 3410.9, 4352.0
1756407600.0, 3410.94, 3417.73, 3409.92, 3415.51, 4347.0
1756411200.0, 3415.49, 3418.74, 3414.67, 3418.01, 3914.0
1756414800.0, 3417.98, 3420.25, 3417.3, 3419.99, 3650.0
1756418400.0, 3419.95, 3423.17, 3417.99, 3422.15, 4141.0
1756422000.0, 3422.32, 3422.4, 3416.41, 3417.36, 2608.0
1756429200.0, 3417.06, 3417.97, 3415.55, 3416.9, 1759.0
1756432800.0, 3416.82, 3416.82, 3414.83, 3414.83, 1513.0
1756436400.0, 3414.84, 3

In [163]:
print("1Hr last candle:", epoch_to_time(last_1h[0], timezone="UTC"))
print("4Hr last candle:", epoch_to_time(last_4h[0], timezone="UTC"))
print("1D last candle:", epoch_to_time(last_1d[0], timezone="UTC"))

1Hr last candle: 2025-09-03 20:00:00
4Hr last candle: 2025-09-03 12:00:00
1D last candle: 2025-09-02 00:00:00
